# Chapter 13: Loading and Preprocessing Data with TensorFlow

In [5]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras

Since Deep Learning systems are often trained on very large datasets that will not fit in RAM, TensorFlow's **Data API** solves this issue by taking care of all the implementation details and only needs:
- A dataset object
- Where to get the data
- How to transform it

## 13.1 The Data API

The Data API revolves around the concept of a **dataset**: a sequence of data items.

In [ ]:
# Create a dataset entirely in RAM
X = tf.range(10) # any data tensor
dataset = tf.data.Dataset.from_tensor_slices(X)
dataset

<TensorSliceDataset shapes: (), types: tf.int32>

The `from_tensor_slices()` function takes a tensor and creates a `tf.data.Dataset` whose elements are all the slices of X. This is the same as `tf.data.Dataset.range(10)`.

In [ ]:
# Iterate over dataset's items
for item in dataset:
    print(item)

tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(1, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(3, shape=(), dtype=int32)
tf.Tensor(4, shape=(), dtype=int32)
tf.Tensor(5, shape=(), dtype=int32)
tf.Tensor(6, shape=(), dtype=int32)
tf.Tensor(7, shape=(), dtype=int32)
tf.Tensor(8, shape=(), dtype=int32)
tf.Tensor(9, shape=(), dtype=int32)


### 13.1.1 Chaining Transformations

Once you have a dataset, you can apply transformations by calling its transformation methods.

In [ ]:
# See Figure 13-1. Chaining dataset transformations
dataset = dataset.repeat(3).batch(7)
for item in dataset:
    print(item)

tf.Tensor([0 1 2 3 4 5 6], shape=(7,), dtype=int32)
tf.Tensor([7 8 9 0 1 2 3], shape=(7,), dtype=int32)
tf.Tensor([4 5 6 7 8 9 0], shape=(7,), dtype=int32)
tf.Tensor([1 2 3 4 5 6 7], shape=(7,), dtype=int32)
tf.Tensor([8 9], shape=(2,), dtype=int32)


With original dataset,
1. Call `repeat(3)` to return a new dataset with 3 copies of the dataset.
    - Calling with no arguments will result in a new dataset that repeats forever, so the code that iterates over the dataset must decide when to stop.

2. Call `batch(7)` to return a new dataset that groups the items into batches of 7 items and any remaining items in the last batch (batch of 2).
    - Add `drop_remainder=True` argument to drop this final batch.

> Note: Dataset methods **do not** modify datasets; they create new ones (ie. assign with `dataset = ...`) or else nothing will happen.

In [ ]:
# Creates new dataset with all items doubled
dataset = dataset.map(lambda x: x * 2) # Items:[0,2,4,6,8,10,12]
for item in dataset:
    print(item)

tf.Tensor([ 0  2  4  6  8 10 12], shape=(7,), dtype=int32)
tf.Tensor([14 16 18  0  2  4  6], shape=(7,), dtype=int32)
tf.Tensor([ 8 10 12 14 16 18  0], shape=(7,), dtype=int32)
tf.Tensor([ 2  4  6  8 10 12 14], shape=(7,), dtype=int32)
tf.Tensor([16 18], shape=(2,), dtype=int32)


While the `map()` method applies a transformation to each item, the `apply()` method applies a transformation to the dataset as a whole.

> Note: `apply()` method is not used since `tf.data.Dataset.unbatch(dataset)` needs 1 argument for the dataset.

In [ ]:
# tf.data.experimental.unbatch() is now deprecated
# Use tf.data.Dataset.unbatch()
# Each item in the new dataset will be single-integer tensor
dataset = tf.data.Dataset.unbatch(dataset)

# Filter the dataset
dataset = dataset.filter(lambda x: x < 10) # Items: 0 2 4 6 8 0 2 4 6...

# Look at just a few items from dataset
for item in dataset.take(3):
    print(item)

tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(4, shape=(), dtype=int32)


### 13.1.2 Shuffling the Data

Gradient Descent works best when the instances in the training set are independent and identically distributed (iid). A simple way to ensure this is to shuffle the instances, using the `shuffle()` method.

> Note: You must specify the buffer size, and it's important to make it large enough, or else shuffling will not be very effective.

> Note: By default, calling `repeat()` on a shuffled dataset will generate a new order at every iteration.

In [2]:
dataset = tf.data.Dataset.range(10).repeat(3) # 0 to 9, three times
dataset = dataset.shuffle(buffer_size=5, seed=42).batch(7)
for item in dataset:
    print(item)

tf.Tensor([0 2 3 6 7 9 4], shape=(7,), dtype=int64)
tf.Tensor([5 0 1 1 8 6 5], shape=(7,), dtype=int64)
tf.Tensor([4 8 7 1 2 3 0], shape=(7,), dtype=int64)
tf.Tensor([5 4 2 7 8 9 9], shape=(7,), dtype=int64)
tf.Tensor([3 6], shape=(2,), dtype=int64)


For large datasets, simple shuffling-buffer may not be enough; buffer size is still small compared to the large dataset.

Some solutions are:
- Shuffle the source data itself.
- Split the source data into multiple files, then read them in a random order during training.
- Pick multiple files randomly and read them simultaneously, interleaving their records.
- Add a shuffling buffer on top of all that using `shuffle()` method.

#### Interleaving lines from multiple files

In [3]:
# FROM TEXTBOOK NOTEBOOK
# Load CA housing dataset
# Split into training, validation, test set
# Scale sets

from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

housing = fetch_california_housing()
X_train_full, X_test, y_train_full, y_test = train_test_split(
    housing.data, housing.target.reshape(-1, 1), random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(
    X_train_full, y_train_full, random_state=42)

scaler = StandardScaler()
scaler.fit(X_train)
X_mean = scaler.mean_
X_std = scaler.scale_

In [6]:
# FROM TEXTBOOK NOTEBOOK
# Split housing set
# Save into 20 CSV files

def save_to_multiple_csv_files(data, name_prefix, header=None, n_parts=10):
    housing_dir = os.path.join("datasets", "housing")
    os.makedirs(housing_dir, exist_ok=True)
    path_format = os.path.join(housing_dir, "my_{}_{:02d}.csv")

    filepaths = []
    m = len(data)
    for file_idx, row_indices in enumerate(np.array_split(np.arange(m), n_parts)):
        part_csv = path_format.format(name_prefix, file_idx)
        filepaths.append(part_csv)
        with open(part_csv, "wt", encoding="utf-8") as f:
            if header is not None:
                f.write(header)
                f.write("\n")
            for row_idx in row_indices:
                f.write(",".join([repr(col) for col in data[row_idx]]))
                f.write("\n")
    return filepaths

train_data = np.c_[X_train, y_train]
valid_data = np.c_[X_valid, y_valid]
test_data = np.c_[X_test, y_test]
header_cols = housing.feature_names + ["MedianHouseValue"]
header = ",".join(header_cols)

train_filepaths = save_to_multiple_csv_files(train_data, "train", header, n_parts=20)
valid_filepaths = save_to_multiple_csv_files(valid_data, "valid", header, n_parts=10)
test_filepaths = save_to_multiple_csv_files(test_data, "test", header, n_parts=10)

In [7]:
# FROM TEXTBOOK NOTEBOOK
# Read 1st few lines of CSV file in text mode

with open(train_filepaths[0]) as f:
    for i in range(5):
        print(f.readline(), end="")

MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedianHouseValue
3.5214,15.0,3.0499445061043287,1.106548279689234,1447.0,1.6059933407325193,37.63,-122.43,1.442
5.3275,5.0,6.490059642147117,0.9910536779324056,3464.0,3.4433399602385686,33.69,-117.39,1.687
3.1,29.0,7.5423728813559325,1.5915254237288134,1328.0,2.2508474576271187,38.44,-122.98,1.621
7.1736,12.0,6.289002557544757,0.9974424552429667,1054.0,2.6956521739130435,33.55,-117.7,2.621


In [8]:
train_filepaths

['datasets/housing/my_train_00.csv',
 'datasets/housing/my_train_01.csv',
 'datasets/housing/my_train_02.csv',
 'datasets/housing/my_train_03.csv',
 'datasets/housing/my_train_04.csv',
 'datasets/housing/my_train_05.csv',
 'datasets/housing/my_train_06.csv',
 'datasets/housing/my_train_07.csv',
 'datasets/housing/my_train_08.csv',
 'datasets/housing/my_train_09.csv',
 'datasets/housing/my_train_10.csv',
 'datasets/housing/my_train_11.csv',
 'datasets/housing/my_train_12.csv',
 'datasets/housing/my_train_13.csv',
 'datasets/housing/my_train_14.csv',
 'datasets/housing/my_train_15.csv',
 'datasets/housing/my_train_16.csv',
 'datasets/housing/my_train_17.csv',
 'datasets/housing/my_train_18.csv',
 'datasets/housing/my_train_19.csv']

In [9]:
# Create dataset with only these file paths
filepath_dataset = tf.data.Dataset.list_files(train_filepaths, seed=42)

> Note: By default, the `list_files()` function returns a dataset that shuffles the file paths.

Next, call `interleave()` method to read from five files at a time and interleave their lines (skipping 1st line which is the header row).

In [10]:
n_readers = 5
dataset = filepath_dataset.interleave(
    lambda filepath: tf.data.TextLineDataset(filepath).skip(1),
    cycle_length=n_readers)

At this stage, there will be 7 datasets in all:
- The filepath dataset
- The interleave dataset
- 5 `TextLineDatasets` created internally by the interleave dataset.

When iterating over the interleave set, it will cycle through these 5 `TextLineDatasets` reading one line at a time until all datasets are out of items. Then it will get the next 5 file paths from `filepath_dataset` and interleave them the same way until it runs out of file paths.

By default, `interleave()` does not use parallelism; it just reads one line at a time from each file. To read files in parallel, set `num_parallel_calls` argument to the number of threads you want.

In [11]:
# 1st rows of 5 CSV files, chosen randomly

for line in dataset.take(5):
    print(line.numpy())

b'4.2083,44.0,5.323204419889502,0.9171270718232044,846.0,2.3370165745856353,37.47,-122.2,2.782'
b'4.1812,52.0,5.701388888888889,0.9965277777777778,692.0,2.4027777777777777,33.73,-118.31,3.215'
b'3.6875,44.0,4.524475524475524,0.993006993006993,457.0,3.195804195804196,34.04,-118.15,1.625'
b'3.3456,37.0,4.514084507042254,0.9084507042253521,458.0,3.2253521126760565,36.67,-121.7,2.526'
b'3.5214,15.0,3.0499445061043287,1.106548279689234,1447.0,1.6059933407325193,37.63,-122.43,1.442'


### 13.1.3 Preprocessing the Data

In [13]:
# X_mean, X_std = # mean and scale of each feature in the training set
# X_mean, X_std already assigned when loading CA dataset

n_inputs = 8

def preprocess(line):
    defs = [0.] * n_inputs + [tf.constant([], dtype=tf.float32)]
    fields = tf.io.decode_csv(line, record_defaults=defs)
    x = tf.stack(fields[:-1])
    y = tf.stack(fields[-1:])
    return (x - X_mean) / X_std, y

Code explanation:

1. (Line 1): Precomputed the mean and standard deviation of each feature in the training set.

2. (Lines 6-8): The `preprocess()` function takes one CSV line and starts by parsing it.
    - (Line 7): All feature columns are floats and missing values should default to 0. And provide an empty array of type `tf.float32` as the default value for the last column (the target).
    - (Line 8): `tf.io.decode_csv()` function takes 2 arguments: the line to parse and an array containing the default value for each column in the CSV file.

3. (Lines 9-10): `decode_csv()` function returns a list of scalar tensors (1 per column) but we need to return 1D tensor arrays.
    - (Line 9): Call `tf.stack()` on all tensors except last one (the target) to stack tensors into a 1D array.
    - (Line 10): Call `tf.stack()` on target value to stack into a 1D tensor array with a single value, rather than a scalar tensor.

4. (Line 11): Scale the input features by subtracting the feature means `(x - X_mean)`, dividing by the feature standard deviation `/ X_std`, and return a tuple containing the scaled features and the target.


In [14]:
# Test preprocessing function
preprocess(b'4.2083,44.0,5.323204419889502,0.9171270718232044,846.0,2.3370165745856353,37.47,-122.2,2.782')

(<tf.Tensor: shape=(8,), dtype=float32, numpy=
 array([ 0.16579159,  1.216324  , -0.05204396, -0.39210168, -0.5277444 ,
        -0.26334172,  0.8543046 , -1.3072058 ], dtype=float32)>,
 <tf.Tensor: shape=(1,), dtype=float32, numpy=array([2.782], dtype=float32)>)

### 13.1.4 Putting Everything Together

### 13.1.5 Prefetching

### 13.1.6 Using the Dataset with tf.keras

## 13.2 The TFRecord Format

### 13.2.1 Compressed TFRecord Files

### 13.2.2 A Brief Introduction to Protocol Buffers

### 13.2.3 TensorFlow Protobufs

### 13.2.4 Loading and Parsing Examples

### 13.2.5 Handling Lists of Lists Using the SequenceExample Protobuf

## 13.3 Preprocessing the Input Features

### 13.3.1 Encoding Categorical Features Using One-Hot Vectors

### 13.3.2 Encoding Categorical Features Using Embeddings

### 13.3.3 Keras Preprocessing Layers

## 13.4 TF Transform

## 13.5 The TensorFlow Datasets (TFDS) Project